<a href="https://colab.research.google.com/github/ttk66/Chem_analyzis/blob/main/si50_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Загрузка данных
df = pd.read_csv("/content/final_filtered_data.csv")
X = df.drop(columns=["IC50", "CC50", "SI", "log_IC50", "log_CC50", "log_SI"])
y = df["log_SI"]

# Масштабирование
X_scaled = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Модели и параметры
model_configs = {
    "Linear Regression": (LinearRegression(), {}),
    "Ridge": (Ridge(), {"alpha": [0.1, 1.0, 10.0]}),
    "Lasso": (Lasso(), {"alpha": [0.001, 0.01, 0.1, 1.0]}),
    "Random Forest": (RandomForestRegressor(random_state=42), {
        "n_estimators": [100, 200],
        "max_depth": [None, 10, 20]
    }),
    "Gradient Boosting": (GradientBoostingRegressor(random_state=42), {
        "n_estimators": [100, 200],
        "learning_rate": [0.05, 0.1],
        "max_depth": [3, 5]
    }),
    "HistGradientBoosting": (HistGradientBoostingRegressor(random_state=42), {
        "learning_rate": [0.05, 0.1],
        "max_iter": [100, 200]
    }),
    "XGBoost": (XGBRegressor(random_state=42, objective='reg:squarederror'), {
        "n_estimators": [100, 200],
        "max_depth": [3, 6],
        "learning_rate": [0.05, 0.1]
    }),
}
# Обучение и оценка
for name, (model, params) in model_configs.items():
    if params:
        model = GridSearchCV(model, params, cv=3, scoring="neg_root_mean_squared_error", n_jobs=-1)
    model.fit(X_train, y_train)
    y_pred_log = model.predict(X_test)

    print(f"\n{name}")
    if isinstance(model, GridSearchCV):
        print("Лучшие параметры:", model.best_params_)
    print("RMSE (лог-шкала):", np.sqrt(mean_squared_error(y_test, y_pred_log)))
    print("R² (лог-шкала):", r2_score(y_test, y_pred_log))


Linear Regression
RMSE (лог-шкала): 1.5629909709181435
R² (лог-шкала): -0.009476605794399351

Ridge
Лучшие параметры: {'alpha': 10.0}
RMSE (лог-шкала): 1.429893339919953
R² (лог-шкала): 0.15512860360637415

Lasso
Лучшие параметры: {'alpha': 0.01}
RMSE (лог-шкала): 1.408040391233221
R² (лог-шкала): 0.18075547658988633

Random Forest
Лучшие параметры: {'max_depth': 10, 'n_estimators': 200}
RMSE (лог-шкала): 1.2703266513913338
R² (лог-шкала): 0.333171520580296

Gradient Boosting
Лучшие параметры: {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100}
RMSE (лог-шкала): 1.3147986437332093
R² (лог-шкала): 0.2856651881044503

HistGradientBoosting
Лучшие параметры: {'learning_rate': 0.05, 'max_iter': 100}
RMSE (лог-шкала): 1.2923242487161564
R² (лог-шкала): 0.3098773089786032

XGBoost
Лучшие параметры: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}
RMSE (лог-шкала): 1.3190838819102235
R² (лог-шкала): 0.28100122950383255
